# Prepare data in 3D

One dataset for each of the three Ziehtiefe (30/50/70) is generated.

In [1]:
import os
import sys
import numpy as np
import pandas as pd

The files ztXX.h5 are on the owncloud and should be put in ../data.

In [8]:
zt_id = 'zt30'
zt_id = 'zt50'
zt_id = 'zt70'

if zt_id == 'zt30':
    h5file = '../data/zt30.h5'
elif zt_id == 'zt50':
    h5file = '../data/zt50.h5'
elif zt_id == 'zt70':
    h5file = '../data/zt70.h5'

with pd.HDFStore(h5file) as store:
    zt = store.get("data")
    elcenter3d = store.get("elcenter3d")

len(elcenter3d)

31976

In [9]:
zt


,Niederhalterkraft,Einlegeposition,Material_ID,Stempel_ID,deviation_e1,deviation_e2,deviation_e3,deviation_e4,deviation_e5,deviation_e6,...,thickness_e31968,thickness_e31969,thickness_e31970,thickness_e31971,thickness_e31972,thickness_e31973,thickness_e31974,thickness_e31975,thickness_e31976,zt
Id,,,,,,,,,,,,,,,,,,,,,
3,210,-1,1,3,-0.207341,-0.074941,-0.107882,-0.029167,0.042297,-0.067442,...,0.957809,0.958256,0.957938,0.961097,0.959042,0.967162,0.963350,0.983586,0.954949,70.0
8,200,2,4,3,0.649186,0.786831,0.695726,0.812256,0.855989,0.721439,...,0.976119,0.976966,0.971750,0.967721,0.968565,0.973183,0.967137,0.988303,0.959734,70.0
10,240,1,6,2,0.182987,0.236061,0.183240,0.260210,0.306625,0.183691,...,0.965734,0.966149,0.964740,0.966794,0.965589,0.971380,0.966934,0.987416,0.958858,70.0
16,220,5,2,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,70.0
18,440,2,6,2,0.182828,0.292917,0.249238,0.314380,0.337067,0.271484,...,0.964622,0.966846,0.963721,0.960945,0.963213,0.967365,0.961011,0.983004,0.955197,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,340,5,5,3,0.595310,0.720982,0.648444,0.763349,0.841330,0.684627,...,1.448482,1.453736,1.447791,1.434766,1.436568,1.447103,1.428275,1.468363,1.422573,70.0
986,440,-2,4,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,70.0
992,320,4,6,2,0.048948,0.125448,0.113846,0.151603,0.219083,0.139836,...,0.967223,0.970495,0.970410,0.964673,0.962748,0.973406,0.961442,0.982747,0.958123,70.0


Save the geometry (xyz coordinates of each node)

In [10]:
geometry = pd.DataFrame(columns=['node', 'x', 'y', 'z'])

for el in elcenter3d.index:
    node = int(el[1:])
    x, y, z = elcenter3d.loc[el].to_numpy()
    geometry = pd.concat([geometry, pd.DataFrame([[node, x, y, z]], columns=['node', 'x', 'y', 'z'])])


geometry.to_csv(f'../data/geometry_{zt_id}.csv', index=False)
geometry

,node,x,y,z
0,1,176.863367,7.600876,-68.322733
0,2,176.248467,7.848119,-68.645467
0,3,174.371633,8.462206,-68.322733
0,4,175.008567,8.276696,-68.645467
0,5,172.500267,9.133440,-68.645467
...,...,...,...,...
0,31972,90.337887,-2.260707,0.000000
0,31973,88.838437,1.417456,0.000000
0,31974,88.796363,2.543633,0.000000
0,31975,87.985487,0.844680,0.000000


Filter out the relevant columns

In [11]:
col_input = ['Niederhalterkraft', 'Einlegeposition', 'Material_ID', 'Stempel_ID']
col_deviation = [s for s in zt.columns if 'deviation' in s]
col_thickness = [s for s in zt.columns if 'thickness' in s]

zt_input = zt.filter(col_input)
zt_deviation = zt.filter(col_deviation)
zt_thickness = zt.filter(col_thickness)

# element id start from 1 -> first element id: 1 
# save dataframes in a large csv file
csv_filename = f'../data/{zt_id}_cleaned.csv'

# Create the header
zt_element = pd.DataFrame(columns = col_input + ['X_e','Y_e','Z_e','design_id','element_id','deviation', 'thickness'])

zt_element.to_csv(csv_filename, mode='w')

for element_1 in elcenter3d.index:
    element_1 = int(element_1.replace('e',''))
    
    zt_element = zt_input
    xyz_element = elcenter3d.loc[f'e{element_1}']

    zt_element['X_e'] = xyz_element.center_x
    zt_element['Y_e'] = xyz_element.center_y
    zt_element['Z_e'] = xyz_element.center_z
    zt_element['design_id'] = zt.index
    zt_element['element_id'] = element_1
    
    col_ = col_deviation[element_1-1]
    zt_element['deviation'] = zt_deviation[col_]

    col_ = col_thickness[element_1-1]
    zt_element['thickness'] = zt_thickness[col_]
    
    zt_element.to_csv(csv_filename, mode='a', header=False)
        

It is faster to reload the data and save it in the right format...

In [12]:
df_tmp = pd.read_csv(csv_filename, index_col=False)

df = pd.DataFrame()
df["doe_id"] = df_tmp.design_id
df["node"] = df_tmp.element_id
df["x"] = df_tmp.X_e
df["y"] = df_tmp.Y_e
df["z"] = df_tmp.Z_e
df["deviation"] = df_tmp.deviation
df["thickness"] = df_tmp.thickness

df = df.sort_values(['doe_id', 'node'])
df

,doe_id,node,x,y,z,deviation,thickness
0,3,1,176.863367,7.600876,-68.322733,-0.207341,0.998633
250,3,2,176.248467,7.848119,-68.645467,-0.074941,0.999642
500,3,3,174.371633,8.462206,-68.322733,-0.107882,0.999379
750,3,4,175.008567,8.276696,-68.645467,-0.029167,0.999812
1000,3,5,172.500267,9.133440,-68.645467,0.042297,0.999971
...,...,...,...,...,...,...,...
7992999,1000,31972,90.337887,-2.260707,0.000000,0.026949,1.431105
7993249,1000,31973,88.838437,1.417456,0.000000,0.129812,1.450779
7993499,1000,31974,88.796363,2.543633,0.000000,0.130656,1.433570
7993749,1000,31975,87.985487,0.844680,0.000000,0.186117,1.476954


Remove failed experiments where the deviation is 0 for all nodes.

In [13]:
removed = []

for id in df['doe_id'].unique():

    dev = df[df['doe_id'] == id]['deviation']
    min_dev, max_dev = dev.min(), dev.max()

    if min_dev == 0.0 and max_dev == 0.0:
        df.drop(df[df.doe_id == id].index, inplace=True)
        removed.append(id)

print(removed)

df.to_csv(f'../data/{zt_id}.csv', index=False)

[16, 84, 131, 174, 198, 211, 286, 337, 378, 485, 524, 531, 549, 566, 569, 593, 612, 623, 670, 704, 745, 751, 783, 803, 813, 824, 855, 893, 897, 902, 911, 937, 973, 986]


Concatenate the three datasets

In [3]:
df = pd.read_csv("../data/zt30.csv", index_col=False)
df = pd.concat([df, pd.read_csv("../data/zt50.csv", index_col=False)])
df = pd.concat([df, pd.read_csv("../data/zt70.csv", index_col=False)])
df.to_csv('../data/zt_all_raw.csv', index=False)